In [55]:
import sys
import os
sys.path.append(os.path.abspath("../.."))
from hv_calculation import get_platypus_problem
from hypervolume_jk import GenerationalDistanceMetric

In [57]:
refset = pd.read_csv("./refsets/global_refset.csv")

In [58]:
refset.shape

(2859, 6)

In [59]:
problem = get_platypus_problem()
ei = GenerationalDistanceMetric(refset, problem)

In [60]:
data = pd.read_csv("../output/squared_exponential_rbf/10_hypervolume.csv")
generations = []
for nfe, generation in data.groupby("Unnamed: 0"):
    generation = generation.rename(
        {
            str(i): name
            for i, name in enumerate(
            [
                "hydropower",
                "atomicpowerplant",
                "baltimore",
                "chester",
                "environment",
                "recreation",
            ]
        )
        },
        axis=1,
    )
    generations.append(generation.iloc[:, 2::])

In [61]:
# let's pick one and see what happens
g = generations[10]
print(g.shape)
ei.calculate(g)

(61, 6)


0.03789936473593654

In [62]:
%%timeit
ei.calculate(g)

287 ms ± 3.06 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [63]:
generations[100].shape

(94, 6)

In [64]:
%%timeit
ei.calculate(generations[100])

452 ms ± 9.47 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [65]:
generations[-1].shape

(1333, 6)

In [66]:
%%timeit
ei.calculate(generations[-1])

6.24 s ± 11.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# custom implementation using numpy

In [7]:
math.pow(sum([math.pow(distance_to_nearest(s, self.reference_set), self.d) for s in feasible]), 1.0 / self.d) / len(feasible)



```python
math.pow(sum([math.pow(distance_to_nearest(s, self.reference_set), self.d) for s in feasible]), 1.0 / self.d) / len(feasible)
```


# pseudo code
```
import scipy.spatial.distance as distance

distances = distance.cdist(generation, refset)
minima = np.min(distances, axis=1)
summed_squares = np.sum(np.pow(minima))
gd = math.pow(summed_squares, 1/d) / generation.shape[0]

distance = euclidian distance for solution


```


is it possible to vectorize also over the outer loop?
We basically need to take the 


In [39]:
from sklearn.preprocessing import MinMaxScaler

ref_set = refset.values
generation = g.values

# normalize using the min and max across all generations and reference set
# this is slightly different from how platypus handles it
scaler = MinMaxScaler()
scaler.fit(ref_set)

normalized_refset = scaler.transform(ref_set)
normalized_generation = scaler.transform(generation)

In [49]:
import scipy.spatial.distance as distance
import math



def calculate_ei(normalized_generation, normalized_refset, d=2):
    distances = distance.cdist(normalized_generation, normalized_refset)
    minima = np.min(distances, axis=1)
    summed_squares = np.sum(np.power(minima, d))
    gd = math.pow(summed_squares, 1/d) / normalized_generation.shape[0]   
    
    return gd

In [54]:

g = generations[10]
print(ei.calculate(g))
normalized_generation = scaler.transform(g.values)
calculate_ei(normalized_generation, normalized_refset)

0.03096122396998343


0.03096122396998343

0.03096122396998343

In [28]:
generation = generations[-1]
normalized_generation = scaler.transform(generation.values)

In [29]:
%%timeit
calculate_ei(normalized_generation, normalized_refset)

7.87 ms ± 46.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
